In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

from evaluation import *
from data_loader import *
import random
from datetime import datetime

In [2]:
experiments = range(108, 118)

exp_agent_data = []
exp_station_data = []


for exp_num in experiments:
    config_fname = f'./Exp_{exp_num}/config.yaml'
    
    c = load_config_file(config_fname)
    nn_c = c['nn_hyperparameters']
    federated_c = c['federated_learning_settings']
    algo_c = c['algorithm_settings']
    env_c = c['environment_settings']
    eval_c = c['eval_config']

    ev_info = []

    seed = env_c['seed']

    algorithm_dm = algo_c['algorithm']
    
    def load_from_json_with_error_handling(filepath, columns_specific):
        try:
            return read_csv_data(filepath, columns=columns_specific)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {filepath}: {e.msg} at line {e.lineno}, column {e.colno}")
            return None  # Handle the error and return None or an empty object
    
    
    d_base = f"../../../../storage_1/metrics/Exp_{exp_num}"
    
    if not os.path.exists(d_base):
        d_base = f"../metrics/Exp_{exp_num}"
            
    base_path = f"{d_base}/train/metrics"

    print(f'Loading {base_path}_agent_metrics.csv')
    agent_data = load_from_json_with_error_handling(f'{base_path}_agent_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'agent_index', 'car_model', 'reward', 'distance', 'average_battery'])

    print(f'Loading {base_path}_station_metrics.csv')
    station_data = load_from_json_with_error_handling(f'{base_path}_station_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'station_index', 'traffic'])

    agent_data['seed'] = seed
    agent_data['exp_num'] = exp_num
    agent_data['algorithm'] = algorithm_dm
    agent_data['season'] = env_c['season']

    station_data['seed'] = seed
    station_data['exp_num'] = exp_num
    station_data['algorithm'] = algorithm_dm
    station_data['season'] = env_c['season']

    exp_agent_data.append(agent_data)
    exp_station_data.append(station_data)

Loading ../../../../storage_1/metrics/Exp_108/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_108/train/metrics_station_metrics.csv
Loading ../../../../storage_1/metrics/Exp_109/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_109/train/metrics_station_metrics.csv
Loading ../../../../storage_1/metrics/Exp_110/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_110/train/metrics_station_metrics.csv
Loading ../../../../storage_1/metrics/Exp_111/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_111/train/metrics_station_metrics.csv
Loading ../../../../storage_1/metrics/Exp_112/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_112/train/metrics_station_metrics.csv


FileNotFoundError: [Errno 2] No such file or directory: './Exp_113/config.yaml'

In [ ]:
# Convert data to DataFrame for easier manipulation
df_agent = pd.concat(exp_agent_data, ignore_index=True)
df_station = pd.concat(exp_station_data, ignore_index=True)

# Scale timestep rewards by timestep index (uncomment this for reward v2)
#df['reward'] = df['reward'] / (df['timestep'] + 1)

# Only show a single timestep
#df = df[df['timestep'] == 0]

cumulative_agent_df = (
    df_agent
    .groupby(
        ['episode', 'zone', 'aggregation', 'agent_index', 'seed', 'exp_num', 'algorithm', 'season']
    )[['reward', 'distance', 'average_battery']]
    .sum()
    .reset_index()
)

cumulative_station_df = (
    df_station
    .groupby(
        ['episode', 'zone', 'aggregation', 'station_index', 'seed', 'exp_num', 'algorithm', 'season']
    )[['traffic']]
    .sum()
    .reset_index()
)

# Rename the 'reward' column to 'cumulative_reward' for clarity
cumulative_agent_df.rename(columns={'reward': 'cumulative_reward'}, inplace=True)
cumulative_agent_df.rename(columns={'average_battery': 'energy_used'}, inplace=True)

# Get recalculated episodes using (aggregation number * episodes per aggregation) + episode number
cumulative_agent_df['episode'] = cumulative_agent_df['aggregation'] * nn_c['num_episodes'] + cumulative_agent_df['episode']
cumulative_station_df['episode'] = cumulative_station_df['aggregation'] * nn_c['num_episodes'] + cumulative_station_df['episode']


cumulative_agent_df.head()

In [ ]:
# Assuming the cumulative_reward_df DataFrame is already available
# If it's not, load your dataset into a DataFrame (e.g., pd.read_csv())

# Define a function to calculate the required metrics
def calculate_metrics(agent_data, station_data):
    metrics = []
    algorithms = agent_data['algorithm'].unique()
    
    for algo in algorithms:
        algo_data = agent_data[agent_data['algorithm'] == algo]
        algo_traffic_data = station_data[station_data['algorithm'] == algo]
        
        # Minimum metrics
        min_reward = algo_data['cumulative_reward'].min()
        min_distance = algo_data['distance'].min()
        min_traffic = algo_traffic_data['traffic'].min()
        min_energy = algo_data['energy_used'].min()
        
        # Average metrics
        avg_reward = algo_data['cumulative_reward'].mean()
        avg_distance = algo_data['distance'].mean()
        avg_traffic = algo_traffic_data['traffic'].mean()
        avg_energy = algo_data['energy_used'].mean()
        
        # Maximum metrics
        max_reward = algo_data['cumulative_reward'].max()
        max_distance = algo_data['distance'].max()
        max_traffic = algo_traffic_data['traffic'].max()
        max_energy = algo_data['energy_used'].max()
        
        metrics.append({
            'Algorithm': algo,
            'Min Reward': min_reward,
            'Min Distance': min_distance,
            'Min Traffic': min_traffic,
            'Min Energy': min_energy,
            'Avg Reward': avg_reward,
            'Avg Distance': avg_distance,
            'Avg Traffic': avg_traffic,
            'Avg Energy': avg_energy,
            'Max Reward': max_reward,
            'Max Distance': max_distance,
            'Max Traffic': max_traffic,
            'Max Energy': max_energy
        })
    
    return pd.DataFrame(metrics)

# Calculate the metrics
metrics_df = calculate_metrics(cumulative_agent_df, cumulative_station_df) 

metrics_df.head()

In [ ]:
cumulative_avg_reward_by_algorithm = cumulative_agent_df.groupby(['episode', 'algorithm', 'seed'])['cumulative_reward'].mean().reset_index()
cumulative_avg_reward_by_algorithm['cumulative_reward'] = cumulative_avg_reward_by_algorithm.groupby(['algorithm', 'seed'])['cumulative_reward'].expanding().mean().reset_index(level=[0, 1], drop=True)

In [ ]:
# Plot the cloud plot for each algorithm
plt.figure(figsize=(8, 6))
for algo in cumulative_avg_reward_by_algorithm['algorithm'].unique():
    # Filter the data for the current zone
    algo_data = cumulative_avg_reward_by_algorithm.loc[cumulative_avg_reward_by_algorithm['algorithm'] == algo]
    min_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].min()
    max_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].max()
    mean_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].mean()

    plt.fill_between(
        min_cumulative_avg_reward.index, 
        min_cumulative_avg_reward.values, 
        max_cumulative_avg_reward.values, 
        alpha=0.3
    )
    plt.plot(
        mean_cumulative_avg_reward.index, 
        mean_cumulative_avg_reward.values, 
        label=f'Algorithm {algo} Average Reward'
    )

plt.xlabel('Episode')
plt.ylabel('Cumulative Average Reward')
plt.title(f'Aggs. {federated_c["aggregation_count"]} - Eps. per agg. {nn_c["num_episodes"]} - Cumulative Average Reward per Episode by Algorithm and Seed')
plt.legend()
plt.grid(True)
plt.show()